# Planilha de controle de usuários 

In [1]:
from collections import defaultdict
from django.db.models.signals import *
import pandas as pd
from itertools import chain
from tqdm.notebook import tqdm
import datetime
import calendar
import numpy as np
import openpyxl as op

### Normaliza os meses para passagens de ano

In [2]:
def normalize_month(month, year):
    return month + 12*(year - 2020)

In [3]:
normalize_month(12, 2023)

48

### Verifica se da pra transformar o cnpj em um numero

In [4]:
def is_valid(cnpj):
    try:
        cnpj = int(cnpj.replace('.', '').replace('/', '').replace('-', ''))
        return True
    except:
        return False

In [5]:
def queryset_to_dict(result_list):
    permissoesUsuarios = {}
    for a in result_list:
        try:
            if not permissoesUsuarios.get(a.user.get_full_name()):
                permissoesUsuarios[a.user.get_full_name()] = {}
            if isinstance(permissoesUsuarios.get(a.user.get_full_name()).get(normalize_month(a.history_date.month, a.history_date.year)),list):
                permissoesUsuarios[a.user.get_full_name()][normalize_month(a.history_date.month, a.history_date.year)].append(a.permissions.name)
            elif permissoesUsuarios.get(a.user.get_full_name()).get(normalize_month(a.history_date.month, a.history_date.year)):
                multi_perm_per_month= permissoesUsuarios.get(a.user.get_full_name()).get(normalize_month(a.history_date.month, a.history_date.year))
                permissoesUsuarios[a.user.get_full_name()][normalize_month(a.history_date.month, a.history_date.year)] = [multi_perm_per_month,a.permissions.name]
            else:
                permissoesUsuarios[a.user.get_full_name()][normalize_month(a.history_date.month, a.history_date.year)] = a.permissions.name
        except:
            pass
    return permissoesUsuarios

In [6]:
def cond_ativo_inativo(permissoesUsuarios,mes_referencia):
    usuariosAtivos = []
    usuariosInativos = []
    
    for nome, mes in permissoesUsuarios.items():
        if mes == {}:
            continue
        meses = sorted(list(mes.keys()))
        item = max(list(mes.keys()))
        cargo_atual = mes[item]
        
        if len(meses) > 1:
            if mes[meses[-2]] != 'Inativo':
                cargo_antes_de_inativo = True
            else:
                cargo_antes_de_inativo = False
        else:
            cargo_antes_de_inativo = False
        
    # Se em algum momento do mesmo mês o usuário foi diferente de Inativo ele entra como ativo
        if isinstance(cargo_atual,list):
            if any(cargo for cargo in cargo_atual if cargo !='Inativo') and not any(u for u in usuariosAtivos if nome == u):
                usuariosAtivos.append(nome)     
    # se o cargo atual for diferente de inativo e não for uma lista(o que configura multiplas permissões no mesmo mês) ele é ativo
        elif cargo_atual != 'Inativo' and not isinstance(cargo_atual,list):
            usuariosAtivos.append(nome)
    # Se no mes anterior ao ultimo ele foi diferente de inativo e no ultimo mes foi Inativo e
    # o mes do cargo_atual for igual ao ultimo mes de referencia(ultimo mês pra coleta) e o nome não estiver nos usuariosAtivos
    # ele é ativo, pois se nesse mês ele foi colocado como Inativo 
    # quer dizer que em algum momento do mês de referencia ele foi ativo
        elif cargo_antes_de_inativo and cargo_atual == 'Inativo' and item == mes_referencia and not any(u for u in usuariosAtivos if nome == u):
            usuariosAtivos.append(nome)       
    # se o cargo atual contem a palavra Inativo e não está nos usuariosAtivos então ele é Inativo
        elif 'Inativo' in cargo_atual and nome not in usuariosAtivos:
            usuariosInativos.append(nome)
        else:
            print('Corra pras montanhas')
    
    return usuariosAtivos,usuariosInativos

In [7]:
def permissoes_history_user(result_list,mes_ano):
    temp_ativo,temp_inativo,usuariosAtivos,usuariosInativos=[],[],[],[]
    
    permissoesUsuarios = {}
    
    mes_referencia = normalize_month(mes_ano.month, mes_ano.year)
    companies = list(result_list.values_list('company__name',flat=True).order_by('company__name').distinct('company__name'))
    
    # condições para tratar dos casos de agrupamento de unidades
    if len(companies) == 1:
        permissoesUsuarios = queryset_to_dict(result_list)
        
        usuariosAtivos,usuariosInativos=cond_ativo_inativo(permissoesUsuarios,mes_referencia)
    
    # se há mais de uma unidade quer dizer q foi feito um agrupamentos por unidade e temos que
    # iterar sobre cada unidade separadamente, pra no fim retornar a lista de usuários por causa de um erro de o usuário
    # estar como inativo numa e ativo na outra
    elif len(companies) > 1:
        
        for company in companies:
            result_list_filtered = result_list.filter(company__name=company)
            permissoesUsuarios = queryset_to_dict(result_list_filtered)

            temp_ativo=cond_ativo_inativo(permissoesUsuarios,mes_referencia)[0] + temp_ativo
            temp_inativo=cond_ativo_inativo(permissoesUsuarios,mes_referencia)[1] + temp_inativo
            
        try:
            usuariosInativos = list(set([x for x in temp_inativo if x not in temp_ativo]))
            usuariosAtivos = list(set(temp_ativo))
        except:
            print(usuariosAtivos,usuariosInativos)
    return usuariosAtivos,usuariosInativos

In [8]:
cnpjs=[a for a in Company.objects.filter(
    active=True).exclude(
    cnpj='00000000000000').exclude(name='Entrevias').distinct(
    'cnpj').order_by().values_list(
    'cnpj',flat=True)]+['Entrevias']
companies=cnpjs+[a for a in Company.objects.filter(
    active=True,
    cnpj='00000000000000'
).exclude(
    name__icontains='kartado'
).exclude(
    name='Entrevias'
).order_by(
    'name'
).values_list(
    'name',
    flat=True
)]

In [10]:
companies

['02.297.946/0001-97',
 '02.955.426/0001-24',
 '02.955.426/0002-05',
 '03.222.465/0001-85',
 '05.205.684/0001-81',
 '07.226.794/0001-55',
 '09.313.969/0001-97',
 '09.325.109/0001-73',
 '09.336.431/0001-06',
 '19.521.322/0001-04',
 '19.758.842/0001-35',
 '19.758.842/0004-88',
 '22.221.801/0001-10',
 '26.664.057/0001-89',
 '28.080.762/0001-64',
 '28.256.567/0001-42',
 '34.976.105/0001-00',
 '36.146.575/0001-64',
 '37.456.439/0001-33',
 '46.791.721/0001-68',
 '47.155.252/0001-53',
 '47.815.827/0001-17',
 '49.314.049/0001-08',
 '52.724.344/0001-47',
 '92.779.503/0001-25',
 'Entrevias',
 'Arteris Fernão Dias',
 'Arteris Fernão Dias - Conserva',
 'Arteris Fluminense - Conserva',
 'Arteris Litoral Sul - Conserva',
 'Arteris Litoral Sul - Faixa de Domínio',
 'Arteris Planalto Sul - Conserva',
 'Arteris Planalto Sul - Faixa de Domínio',
 'Arteris Régis Bittencourt - Conserva',
 'Arteris Régis Bittencourt - Faixa de Domínio',
 'Arteris Via Paulista - Faixa de Domínio',
 'Arteris Via Paulista - S

In [22]:
# companies=['ABAPA Demonstração','Entrevias']

In [12]:
Company.objects.get(cnpj='05847054000100')

DoesNotExist: Company matching query does not exist.

In [12]:
# companies=list(Company.objects.filter(name__icontains='Construtora Castilho').values_list('name',flat=True))
# companies

### Mês e ano para coleta dos usuários

In [14]:
# Mês e ano para coleta dos usuários
mes=4
ano=2024
dia = calendar.monthrange(ano, mes)[1]
dia,mes,ano

(30, 4, 2024)

In [15]:
# DATA sempre referente ao mês que se quer coletar nesse formato history_date__lte='2024-01-31 03:00:00'
# é utilizado 3 horas da manha pois é a pra fechar com o horario de meio noite do servidor
data = datetime.datetime(ano,mes,dia,3,0,0)
data

datetime.datetime(2024, 4, 30, 3, 0)

### Seta o nome do arquivo e o cria

In [16]:
filename=f'Usuários Ativos Financeiro_por_CNPJ_{mes}_{ano}'
pd.DataFrame().to_excel(filename+'.xlsx')

### Geração da planilha com os dados de usuários Ativos

In [23]:
for company in tqdm(companies):
    planilha_name=company.replace('.', '').replace('/', '').replace('-', '')
    if is_valid(company):
        # Agrupa por CNPJ
        historicoUsuarios=UserInCompany.history.model.objects.filter(
                company__cnpj=company,
                history_date__lte=data
            ).exclude(
                user__email__icontains='kartado'
            ).exclude(
                user__email__icontains='roadlabs'
            ).exclude(
                user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
            ).exclude(
                user__email__icontains='hermes'
            ).order_by( 'history_date')
    elif 'Entrevias' in company:
        planilha_name="AJR"
        historicoUsuarios=UserInCompany.history.model.objects.filter(
            Q(user__email__icontains='ajr')|Q(user__username__icontains='ajr'),
            company__name__icontains=company,
            history_date__lte=data
        ).exclude(
            user__email__icontains='kartado'
        ).exclude(
            user__email__icontains='roadlabs'
        ).exclude(
            user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
        ).exclude(
            user__email__icontains='hermes'
        ).order_by( 'history_date')
    else:
        historicoUsuarios=UserInCompany.history.model.objects.filter(
                company__name=company,
                history_date__lte=data
            ).exclude(
                user__email__icontains='kartado'
            ).exclude(
                user__email__icontains='roadlabs'
            ).exclude(
                user__uuid = '6ca2ac08-7ee0-4587-a0cf-8850f72e1b95'
            ).exclude(
                user__email__icontains='hermes'
            ).order_by( 'history_date')
    
    users_permissions = permissoes_history_user(historicoUsuarios,data)[0] if len(permissoes_history_user(historicoUsuarios,data)[0]) > 0 else []
    print(permissoes_history_user(historicoUsuarios,data)[0])
    if users_permissions != 0:
        df2 = pd.DataFrame(data=users_permissions,columns=['Usuários ativos'])
        companies_list=historicoUsuarios.distinct('company__name').order_by( 'company__name').values_list('company__name',flat=True)
        print(company,companies_list)
        df2['Companies'] = np.nan
        df2.loc[:len(companies_list)-1, 'Companies'] = companies_list
        with pd.ExcelWriter(filename+'.xlsx', mode='a',if_sheet_exists='replace') as writer:
            df2.to_excel(writer,sheet_name=planilha_name,na_rep="-",index=False)

  0%|          | 0/2 [00:00<?, ?it/s]

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalUserInCompany.history_date received a naive datetime (2024-04-30 03:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalUserInCompany.history_date received a naive datetime (2024-04-30 03:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


['Larissa Silva Campos', 'Paulo Lopes']
ABAPA Demonstração <QuerySet ['ABAPA Demonstração']>
['Isabelly AJR', 'Rogério Buchala', 'Cassio Borges', 'Luciano Teixeira', 'Osmar Duprat', 'Marcio Celestino']
Entrevias <QuerySet ['Entrevias']>


## Geração do Resumo da página com o total de usuários por unidade ou grupo

In [24]:
excel_file = pd.ExcelFile(filename+'.xlsx')
dados=[]
# Iterar sobre as abas
for sheet_name in excel_file.sheet_names:
    df = excel_file.parse(sheet_name)
    
    if 'Usuários ativos' in df.columns:
        tamanho_coluna = len(df['Usuários ativos'])
        dados.append((sheet_name,tamanho_coluna))


In [25]:
wb=op.load_workbook(filename+'.xlsx')

In [26]:
wb=op.load_workbook(filename+'.xlsx')
nome_aba = 'Resumo'
wb['Sheet1'].title = nome_aba
wb.active = wb['Resumo']


wb.save(filename+'.xlsx')

In [27]:
wb.active = wb['Resumo']
ws = wb.active

In [28]:
ws['A1'] = "Unidade"
ws['B1'] = "Quantidade de Usuários"
for i,dado in enumerate(dados,start=2):
    ws['A'+str(i)] = dado[0]
    ws['B'+str(i)] = dado[1]
wb.save(filename+'.xlsx')